In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
import plotly.express as px
import matplotlib.pyplot as plt
from json import loads

CENTER_BARCELONA = {"lat": 41.3851, "lon": 2.1734}

def convert_wkt_to_geometry(df: pd.DataFrame, wkt_column: str) -> gpd.GeoDataFrame:
    # Convert the GEOM_WKT column to geometry
    df['geometry'] = df[wkt_column].apply(wkt.loads)

    # Convert the DataFrame to a GeoDataFrame
    return gpd.GeoDataFrame(df.drop(wkt_column, axis='columns'), geometry='geometry')

In [ ]:
DATA_PATH = "./data/"

gdfs = [
    pd.read_csv(
        DATA_PATH + "air_quality/2023/2023_tramer_no2_mapa_qualitat_aire_bcn.csv"
    )
    .rename(columns={"Rang": "NO2"})
    .astype({"NO2": "category"}),
    pd.read_csv(
        DATA_PATH + "air_quality/2023/2023_tramer_pm2-5_mapa_qualitat_aire_bcn.csv"
    )
    .rename(columns={"Rang": "PM2_5"})
    .astype({"PM2_5": "category"}),
    pd.read_csv(
        DATA_PATH + "air_quality/2023/2023_tramer_pm10_mapa_qualitat_aire_bcn.csv"
    )
    .rename(columns={"Rang": "PM10"})
    .astype({"PM10": "category"}),
]

gdfs = [convert_wkt_to_geometry(gdf, "GEOM_WKT") for gdf in gdfs]
gdf = gdfs[0][["TRAM", "geometry"]]
for temp_gdf in gdfs:
    gdf = gdf.merge(temp_gdf.drop(columns=["geometry"]), on="TRAM")

gdf["NO2"] = gdf["NO2"].cat.reorder_categories(
    [
        "10-20 µg/m³",
        "20-30 µg/m³",
        "30-40 µg/m³",
        "40-50 µg/m³",
        "50-60 µg/m³",
        "60-70 µg/m³",
        ">70 µg/m³",
    ],
    ordered=True,
)

gdf["PM2_5"] = gdf["PM2_5"].cat.reorder_categories(
    ["5-10 µg/m³", "10-15 µg/m³", "15-20 µg/m³", "20-25 µg/m³", "25-30 µg/m³"],
    ordered=True,
)

gdf["PM10"] = gdf["PM10"].cat.reorder_categories(
    [
        "<=15 µg/m³",
        "15-20 µg/m³",
        "20-25 µg/m³",
        "25-30 µg/m³",
        "30-35 µg/m³",
        "35-40 µg/m³",
        "> 40 µg/m³",
    ],
    ordered=True,
)

gdf: gpd.GeoDataFrame = gdf.set_crs(epsg=25831).to_crs(epsg=4326)

In [ ]:
loads(gdf.to_json())

In [ ]:
from shapely.ops import linemerge

# Group by NO2 and merge geometries
merged_gdf = gdf.groupby('NO2', observed=False)['geometry'].apply(func=lambda x: linemerge(x.unary_union)).reset_index()

# Convert to GeoDataFrame
merged_gdf = gpd.GeoDataFrame(merged_gdf, geometry='geometry')

merged_gdf

In [ ]:
lats = []
lons = []
names = []

import shapely.geometry
import numpy as np

for feature, name in zip(merged_gdf.geometry, merged_gdf["NO2"]):
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        names = np.append(names, [name]*len(y))
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        names = np.append(names, None)

fig = px.line_geo(lat=lats, lon=lons, hover_name=names)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
geojson

In [ ]:
df

In [ ]:

df = px.data.election()
geojson = px.data.election_geojson()

fig = px.choropleth_map(df, geojson=geojson, color="Bergeron",
                           locations="district_id",
                           center={"lat": 45.5517, "lon": -73.7073},
                           map_style="carto-positron", zoom=9)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.histogram(
    gdf.value_counts("NO2").reset_index(),
    x="NO2",
    y="count",
    category_orders={"NO2": gdf["NO2"].cat.categories},
    title="Distribution de la fourchette de valeurs de polluants de 2023 à Barcelone",
)
fig = fig.update_layout(
    yaxis_title="Somme des valeurs de polluants",
    xaxis_title="Fourchette de valeurs de polluants, en moyenne annuelle [µg/m3]",
)
fig.show()

In [ ]:
px.colors.sequential.Plasma

In [ ]:
gdf["PM10"].cat.reorder_categories(['<=15 µg/m³', '15-20 µg/m³', '20-25 µg/m³', '25-30 µg/m³', '30-35 µg/m³',
       '35-40 µg/m³', '> 40 µg/m³'], ordered=True)

In [ ]:
import folium

map = folium.Map(
    location=list(CENTER_BARCELONA.values()), tiles="CartoDB Positron", zoom_start=12
)

polluant = "PM10"
# Define a color map for NO2 levels
color_map = dict(zip(gdf[polluant].cat.categories, px.colors.sequential.Plasma_r))

def style_function(feature):
    return {
        'fillColor': color_map.get(feature['properties'][polluant], 'gray'),
        'color': color_map.get(feature['properties'][polluant], 'gray'),
        'weight': 2,
        'fillOpacity': 0.6,
    }

folium.GeoJson(
    gdf.to_json(),
    name="Air Quality",
    tooltip=folium.GeoJsonTooltip(fields=["NO2", "PM2_5", "PM10"], aliases=["NO2", "PM2.5", "PM10"]),
    style_function=style_function
).add_to(map)

# Add legend
legend_html = f'''
<div style="position: fixed; 
            top: 50px; right: 50px; width: 120px; height: {35 + len(gdf[polluant].cat.categories) * 20}px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white;
            padding: 5px 10px;
            ">
    <b>{polluant}</b><br>
    {'<br>'.join(f'<i style="background:{color_map[cat]}">&nbsp;&nbsp;&nbsp;&nbsp;</i> {cat}' for cat in gdf[polluant].cat.categories)}<br>
</div>
'''

map.get_root().html.add_child(folium.Element(legend_html))

map

In [ ]:
px.histogram(
    gdf,
    x="Rang",
    title="Histograma de la qualitat de l'aire a Barcelona",
    labels={"Rang": "Qualitat de l'aire"},
    category_orders={"Rang": gdf["Rang"].cat.categories},
)

In [ ]:
import plotly.io as pio
import dash_mantine_components as dmc

dmc.add_figure_templates(default="mantine_dark")

type(pio.templates['mantine_dark'])

pio

In [ ]:
plt.figure(figsize=(6, 10))
gdf.plot(
    ax=plt.gca(),
    column="NO2",
    legend=True
)

---

In [ ]:
df = pd.concat(
    [
        pd.read_csv(
            filepath_or_buffer="./data/noise_monitoring/2023/2023_1S_XarxaSoroll_EqMonitor_Dades_1Hora.csv",
        ),
        pd.read_csv(
            filepath_or_buffer="./data/noise_monitoring/2023/2023_2S_XarxaSoroll_EqMonitor_Dades_1Hora.csv",
        ),
    ]
)

df["date"] = pd.to_datetime(
    df["Any"].astype(str)
    + "-"
    + df["Mes"].astype(str)
    + "-"
    + df["Dia"].astype(str)
    + " "
    + df["Hora"].astype(str)
)

df = df.drop(columns=["Any", "Mes", "Dia", "Hora"])

gdf = pd.read_csv("./data/noise_monitoring/XarxaSoroll_EquipsMonitor_Instal.csv")
gdf = gdf[gdf["Id_Instal"].isin(df["Id_Instal"].unique())]

In [ ]:
px.line(
    df,
    x="date",
    y="Nivell_LAeq_1h",
    color="Id_Instal",
    title="Nivell de soroll a Barcelona",
)

In [ ]:
fig = px.scatter_mapbox(
    gdf,
    lat="Latitud",
    lon="Longitud",
    hover_name="Nom_Carrer",
    hover_data=["Nom_Barri", "Nom_Districte"],
    color="Font",
    zoom=10,
    height=600
)

fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = px.sunburst(
    df.merge(gdf[["Id_Instal", "Nom_Districte", "Font"]], on="Id_Instal"),
    path=["Nom_Districte", "Font"],
    title="Sunburst of noise type in Barcelona by district",
)
fig.show()

In [ ]:
px.histogram(
    data_frame=df,
    x="Nivell_LAeq_1h",
    marginal="box",
    nbins=250,
)

---

In [ ]:
df = pd.read_csv("./data/district_zone/BarcelonaCiutat_Districtes.csv")

gdf = convert_wkt_to_geometry(df, 'geometria_wgs84')

In [ ]:
fig = px.choropleth_map(
    gdf,
    geojson=gdf.geometry,
    locations=gdf.index,
    color="nom_districte",
    center=CENTER_BARCELONA,
    title="Map of Districts in Barcelona"
)

fig.show()